# Machine Learning: Introduction

I think of machine learning as a set of algorithms and implementations that make data-driven predictions or decisions. This is why machine learning has become all the buzz in industry, but isn't always applicable in scientific studies; we're often interested in making some inference about a parameter after marginalizing over nuisance parameters so we can learn some thing about "physics." We're usually not trying to serve ads to teenagers.

The typical problems for which you might consider using machine learning methods can be grouped into four main types of problems:

* **Classification**: I have photometry for 100,000 sources from some imaging survey and I want to know which ones are galaxies, stars, quasars
* **Regression**: I want to learn the parameters of some model, usually so I can make good predictions.
* **Dimensionality reduction**: I have spectra for 100,000 galaxies and I want to decompose the spectra into a mixture of stellar population models.
* **Clustering**: I have colors, sizes, and shapes for 100,000 galaxies and I want to know if there are sub-populations with similar characteristics.

https://github.com/jakevdp/sklearn_pycon2015/blob/master/notebooks/02.2-Basic-Principles.ipynb